Import Dependencies 

In [1]:
# !git clone https://github.com/tmengel/Deep-Learning.git
# %cd Deep-Learning/finalProject
# %pip install uproot
# %pip install pandas

# from google.colab import drive
# drive.mount('/content/drive')

import pulsenet as pn
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import pandas as pd

2023-05-10 19:03:58.487829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 19:03:59.790950: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tmengel/root/lib
2023-05-10 19:03:59.791060: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-10 19:04:02.721824: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared obj

File name

In [2]:
filename = "ysoTracesWithPileup.root"
# filename = "/content/drive/MyDrive/DeepLearningFinalProject/ysoTracesWithPileup.root"

discriminator_head_weights = "weights/discriminator_head_initial.h5"
discriminator_base_weights = "weights/discriminator_base_initial.h5"
classifier_head_weights = "weights/classifier_head_initial.h5"
classifier_base_weights = "weights/classifier_base_initial.h5"
phase_weights = "weights/phase_initial.h5"
amplitude_weights = "weights/amplitude_initial.h5"
transfer_head_weights = "weights/transfer_head_initial.h5"

discriminator_base_fine_tuned_weights = "weights/discriminator_base_fine_tuned.h5"
classifier_base_fine_tuned_weights = "weights/classifier_base_fine_tuned.h5"
phase_fine_tuned_weights = "weights/phase_fine_tuned.h5"
amplitude_fine_tuned_weights = "weights/amplitude_fine_tuned.h5"
transfer_head_fine_tuned_weights = "weights/transfer_head_fine_tuned.h5"

discriminator_initial_history ="history/discriminator_initial_history.h5"
classifier_initial_history = "history/classifier_initial_history.h5"
phase_initial_history = "history/phase_initial_history.h5"
amplitude_initial_history = "history/amplitude_initial_history.h5"

discriminator_base_control_weights = "weights/discriminator_base_control.h5"
classifier_base_control_weights = "weights/classifier_base_control.h5"
phase_control_weights = "weights/phase_control.h5"
amplitude_control_weights = "weights/amplitude_control.h5"
transfer_head_control_weights = "weights/transfer_head_control.h5"

transfer_history = "history/transfer_history.h5"
fine_tune_history = "history/fine_tune_history.h5"
control_history = "history/control_history.h5"

transfer_model = "weights/transfer_model.h5"
fine_tuned_model = "weights/fine_tuned_model.h5"
control_model = "weights/control_model.h5"

In [6]:
# Train Discriminator
# Get Data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=1.0, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# output files
historyfile = "history/discriminator_initial_history.h5"

# Create model
model = keras.Sequential([
    pn.TraceDiscriminatorBase(name="discriminator_base"),
    pn.TraceDiscriminatorHead(name="discriminator_head")
])
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train model
history = model.fit(x_trace, y_trace, epochs=200, batch_size=128, validation_split=0.2, verbose=1)
model.layers[0].save_weights(discriminator_base_weights)
model.layers[1].save_weights(discriminator_head_weights)
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(discriminator_initial_history, key="hist")

!cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
!cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

Created 261852 samples: 100.0 % pileup, 0.0 % no pileup


In [ ]:
# Train Classifier
# Get Data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
y_pileup = pn.EncodePileup(y_phase)
# Create model
model = keras.Sequential([
    pn.TraceClassifierBase(name="classifier_base"),
    pn.TraceClassifierHead(name="classifier_head")
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(x_trace, y_pileup, epochs=200, batch_size=128, validation_split=0.2, verbose=1)
model.layers[0].save_weights(classifier_base_weights)
model.layers[1].save_weights(classifier_head_weights)
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(classifier_initial_history, key="hist")

!cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
!cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

In [ ]:
# Train Amplitude Regressor
# Get Data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# Create model
model = keras.Sequential([
    pn.TraceAmplitudeRegressor(name = "amplitude_regressor")
])
model.compile(optimizer='adam', loss="mse", metrics=['accuracy'])

# Train model
history = model.fit(x_trace, y_amp, epochs=200, batch_size=128, validation_split=0.2, verbose=1)
model.layers[0].save_weights(amplitude_weights)
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(amplitude_initial_history, key="hist")

!cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
!cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

In [ ]:
# Train Phase Regressor
# Get Data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# Create model
model = keras.Sequential([
    pn.TracePhaseRegressor(name = "phase_regressor")
])
model.compile(optimizer='adam', loss="mse", metrics=['accuracy'])

# Train model
history = model.fit(x_trace, y_phase, epochs=200, batch_size=128, validation_split=0.2, verbose=1)
model.layers[0].save_weights(phase_weights)
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(phase_initial_history, key="hist")

!cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
!cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

In [3]:
input = layers.Input(shape=(1, 300))  # Returns a placeholder tensor
classifer_feature_vec = pn.TraceClassifierBase(name = "classifier_base")(input)
discriminator_feature_vec = pn.TraceDiscriminatorBase(name = "discriminator_base")(input)
trace_output = pn.TraceClassifierDiscriminatorHead(name = "transfer_head")([discriminator_feature_vec, classifer_feature_vec])
phase_output = pn.TracePhaseRegressor(name="phase_regressor")(trace_output)
amplitude_output = pn.TraceAmplitudeRegressor(name ="amplitude_regressor")(trace_output)
classifier_output = pn.TraceClassifierHead(name="classifier_head")(classifer_feature_vec)

In [4]:
# load new data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# create model
model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer='adam', loss=['mse', 'mse', 'mse'], metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights(discriminator_base_weights)
model.layers[model_names.index("classifier_base")].load_weights(classifier_base_weights)
model.layers[model_names.index("phase_regressor")].load_weights(phase_weights)
model.layers[model_names.index("amplitude_regressor")].load_weights(amplitude_weights)
# freeze layers
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("classifier_base")].trainable = False
model.layers[model_names.index("phase_regressor")].trainable = False
model.layers[model_names.index("amplitude_regressor")].trainable = False                           
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, [y_trace, y_phase, y_amp], epochs=200, batch_size=128, validation_split=0.2, verbose=1)
# save weights
model.layers[model_names.index("transfer_head")].save_weights(transfer_head_weights)
# save model
model.save(transfer_model)
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(transfer_history, key="hist")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 300)]     0           []                               
                                                                                                  
 discriminator_base (TraceDiscr  (None, 2, 300)      992358      ['input_1[0][0]']                
 iminatorBase)                                                                                    
                                                                                                  
 classifier_base (TraceClassifi  (None, 1, 300)      244456      ['input_1[0][0]']                
 erBase)                                                                                          
                                                                                              

In [ ]:
# load new data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# create model
model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=['mse', 'mse', 'mse'], metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights(discriminator_base_weights)
model.layers[model_names.index("classifier_base")].load_weights(classifier_base_weights)
model.layers[model_names.index("phase_regressor")].load_weights(phase_weights)
model.layers[model_names.index("amplitude_regressor")].load_weights(amplitude_weights)
model.layers[model_names.index("transfer_head")].load_weights(transfer_head_weights)                     
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, [y_trace, y_phase, y_amp], epochs=200, batch_size=128, validation_split=0.2, verbose=1)
# save weights
model.layers[model_names.index("discriminator_base")].save_weights(discriminator_base_fine_tuned_weights)
model.layers[model_names.index("classifier_base")].save_weights(classifier_base_fine_tuned_weights)
model.layers[model_names.index("phase_regressor")].save_weights(phase_fine_tuned_weights)
model.layers[model_names.index("amplitude_regressor")].save_weights(amplitude_fine_tuned_weights)
model.layers[model_names.index("transfer_head")].save_weights(transfer_head_fine_tuned_weights)
# save model
model.save(fine_tuned_model)
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(fine_tune_history, key="hist")

In [ ]:
# load new data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# create model
model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=['mse', 'mse', 'mse'], metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# print summary
model.summary()
# train control model
history = model.fit(x_trace, [y_trace, y_phase, y_amp], epochs=400, batch_size=128, validation_split=0.2, verbose=1)
# save weights
model.layers[model_names.index("discriminator_base")].save_weights(discriminator_base_control_weights)
model.layers[model_names.index("classifier_base")].save_weights(classifier_base_control_weights)
model.layers[model_names.index("phase_regressor")].save_weights(phase_control_weights)
model.layers[model_names.index("amplitude_regressor")].save_weights(amplitude_control_weights)
model.layers[model_names.index("transfer_head")].save_weights(transfer_head_control_weights)
# save model
model.save(control_model)
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(control_history, key="hist")